In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from nltk.tokenize import sent_tokenize

import math
from collections import Counter

import requests
import json

import pandas as pd
import numpy as np

In [2]:
def tf_idf_calculator(lines):    
    # Compute the term frequency of each word in each sentence
    tf = []
    
    for sentence in lines:
        tf_sentence = {}
        for word in sentence.split():
            tf_sentence[word] = tf_sentence.get(word, 0) + 1
        tf.append(tf_sentence)

    # Compute the inverse document frequency of each word
    idf = {}
    for sentence in lines:
        for word in sentence.split():
            idf[word] = idf.get(word, 0) + 1
    for word in idf:
        idf[word] = math.log(len(lines) / idf[word])

    # Compute the tf-idf of each sentence
    tf_idf = []
    for i, sentence in enumerate(lines):
        tf_idf_sentence = 0
        for word in tf[i]:
            tf_idf_sentence += tf[i][word] * idf[word]
        tf_idf.append(tf_idf_sentence)
    
    return tf_idf

In [3]:
seasons = []

for i in range(1,11):

    # load season i
    if i<10:
        json_file = 'https://raw.githubusercontent.com/emorynlp/character-mining/master/json/friends_season_0'+str(i)+'.json'
    else:
        json_file = 'https://raw.githubusercontent.com/emorynlp/character-mining/master/json/friends_season_10.json'
    
    r = requests.get(json_file)
    seasons.append(json.loads(r.text))

#we can do this manually
characters = ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Ross Geller', 'Rachel Green']


In [4]:
values = []
values_lower = []

for character in characters:
    lines = []

    for season in seasons:
        for episode in season['episodes']:
            for scene in episode['scenes']:
                for utterance in scene['utterances']:
                    if utterance['speakers'] != []:
                        if character in utterance['speakers']:
                            lines.append(utterance['transcript'])

    
    tf_idf = tf_idf_calculator(lines)

    final = []

    #format: final = [...,[sentence, tf-idf score],...]
    for i,sentence in enumerate(lines):
        cell = []
        cell= [tf_idf[i], sentence]
        final.append(cell)
    
    # lower -> higher     
    sorted_list = sorted(final)

    l = []

    #get only the strings
    for elem in sorted_list:
        l.append(elem[1])

    #remove duplicates
    l = list( dict.fromkeys(l) )
    
    counts = l[-15:]
    lower = l[:20]

    higher = l[-100:]

    f = open(character + ".txt", "w")
    for line in higher:
	    f.write(line+"\n")
    f.close()

    values.append(counts)
    values_lower.append(lower)

In [5]:
dict = {key: value for key, value in zip(characters, values)}

df = pd.DataFrame.from_dict(dict)

# displaying the DataFrame
df.style

,Monica Geller,Joey Tribbiani,Chandler Bing,Phoebe Buffay,Ross Geller,Rachel Green
0,"No, look, she's obviously unstable, okay? I mean she's thinking about running out on her wedding day. Okay, fine! But I mean, look at the position she's putting him in! What's he gonna do? Ross is gonna run over there on the wedding day and break up the marriage?! I mean, who would do that?! Okay, fine, all right, but that's y'know, it's different! Although it did involve a lot of the same people.","Now-now, listen this is just a first draft so... ""We are gathered here today on this joyous occasion to celebrate the special love that Monica and Chandler share."" Eh? ""It is a love based on giving and receiving. As well as having and sharing. And the love that they give and have is shared and received. And through this having and giving and sharing and receiving."" ""We too can share and love and have and receive.""","Oh come on guys, it's not like I moved to Europe! I just moved across the hall! And we would have you over all the time if it weren't for Monica's allergies. You're right, I could never lie to you. She hates you. Should I get that? Hello? Uh no, Joey's not here right now. Can I take a message? Yeah, okay so the audition has been moved from 5:00 to 2:30? Okay great. Bye.","There was a girl, we'll call her Betty, and a guy let's call him Neil. Now I can't stress this point too strongly, this story isn't real. Now our Neil must decide, who will be the girl that he casts aside. Will Betty be the one who he loves truly? Or will it be the one who we'll call Ju...Loolie? He must decide, he must decide, even though I made him up, he must decide!","Uh-huh, Carol, so were we. All right, just-just imagine for a moment, Susan meets someone and-and they really hit it off. Y'know? Say-say they're coming back from the theatre, and they-they stop at a pub for a couple of drinks, they're laughing, y'know, someone innocently touches someone else... There's electricity, it's new. It's exciting. Are you telling me there isn't even the slightest possibility of something happening?","What? This-this, no, oh no, no-no-no, this is not-that's-that's not what it is. See, see, okay, I work in fashion, see and-and, this is a real dress actually. It's-it's, they're-they're wearing it in Milan, so part of my job is too wear the clothes, and then I see how people respond, and then I report back to my superiors at Bloomingdale's, so... And obviously in uh, in-in this case, I am going to report back, ""USA not ready."""
1,"No. He Teaches A Course On Food Criticism At The New School, So Before We Go To The Movies I Wanna Go By There And Make Him Try My Bouillabaisse Again. Oh, I Cannot Wait To Read The Front Page Of The Post Tomorrow! ""Restaurant Reviewer Admits I was wrong about Monica.""","Well, I've never been through the tunnel myself, 'cause as I understand it, you're not allowed to go through with more than one girl in the car, right. But, it seems to me it's pretty much like anything else, you know, face your fear. It have a fear of heights, you go to the top of the building! If you're afraid of bugs.....get a bug. Right. In this case, you have a fear of commitment, so I say you go in there and be the most committed guy there ever was.","Okay, what do you saw I go over there and say how much I like her? No-no it'll be good, I can tell her much I've been thinking about her. That I haven't stopped thinking about her since the moment I met her. That I'm so fantastically, over-the-top, wanna-slit-my-own-throat in love with her, that for every minute of every hour of every day I can't believe my own damn bad luck that you met her first!!","OK, um, I'm gonna play, um, some songs about grandparents, OK. Now, grandma's a person who everyone likes, she bought you a train and a bright, shiny bike. But lately she hasn't been coming to dinner, And last time you saw her she looked so much thinner. Now, your mom and your dad said she moved to Peru, but the truth is she

In [6]:
dict = {key: value for key, value in zip(characters, values_lower)}

df = pd.DataFrame.from_dict(dict)

# displaying the DataFrame
df.style

,Monica Geller,Joey Tribbiani,Chandler Bing,Phoebe Buffay,Ross Geller,Rachel Green
0,,,Hey!,,,
1,Okay.,Hey!,Okay.,Hey!,What?,Okay.
2,Hey!,Yeah.,What?,Okay.,Hey!,What?
3,What?,Yeah!,Hey.,Oh!,Yeah.,what?
4,Yeah.,Okay.,No!,Yeah.,Okay.,Hi!
5,No!,What?,.,God!,No!,Oh!
6,What?!,Hey.,Yeah.,No!,Hey,Yeah.
7,No.,No,Okay!,Yeah!,No.,God!
8,Hey.,No!,Really?,Oh.,Okay?,Hey!
9,Okay!,Yeah,No.,Hey.,Yeah!,No!
